# Condo Ownership vs Stock Index Investment

In [339]:
import numpy as np

## Yearly Expenditures for Condo Ownership (3494 hotel-de-ville)

(Property Tax for [Plateau Mont-Royale](http://ville.montreal.qc.ca/pls/portal/docs/PAGE/SERVICE_FIN_EN/MEDIA/DOCUMENTS/2019_PLATEAU_ANG.PDF)) * (Most recent property evaluation for 3494 hotel-de-ville)

[School Tax](https://www.cgtsim.qc.ca/en/8-english-canada/173-2017-school-tax-2)


In [340]:
plateau_tax_rate = 0.006519 + 0.000025 + 0.001083 + 0.000036 + 0.000591 + 0.000315
school_tax = 0.0017832
municipal_evaluation = 359000
PROPERTY_TAX = (plateau_tax_rate + school_tax) * municipal_evaluation
PROPERTY_TAX

bmo_prop_tax_confirmed=3076.28
bmo_school_tax_confirmed=502.17
PROPERTY_TAX=bmo_prop_tax_confirmed + bmo_school_tax_confirmed

In [341]:
CONDO_FEES = 2400
INSURANCE = 1000
MAINTENANCE = 1000

ANNUAL_EXPENSE = sum([CONDO_FEES, INSURANCE, MAINTENANCE, PROPERTY_TAX])
ANNUAL_EXPENSE

7978.450000000001

## One-time Expenses for Condo Ownership

In [342]:
welcome_tax = 4500
notary = 1500
title_insurance = 400
inspection = 1000
REPAIR=4000

INITIAL_EXPENSE = sum([welcome_tax, notary, title_insurance, inspection, REPAIR])
INITIAL_EXPENSE

11400

## Mortgage Analysis

### CMHC Insurance

Interest rates [listed here](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/mortgage-loan-insurance-homeownership-programs/cmhc-mortgage-loan-insurance-cost). Mortgage insurance is covered by the lender for loan to value of under 80% (i.e., if your downpayment is at least 20%, this section is of no concern to you).

In [343]:
def mortgage_insurance_rate(p, downpayment):
    ltv = (p - downpayment) / p
    assert ltv > 0
    assert ltv <= 1
    
    if ltv >= 0.95:
        return 0.04
    elif ltv >= 0.90:
        return 0.031
    elif ltv >= 0.85:
        return 0.028
    elif ltv >= 0.80:
        return 0.024
    else:
        return 0

In [344]:
def mortgage(price, downpayment, interest, periods):
    initial_principal = price + mortgage_insurance_rate(price, downpayment) * price - downpayment
    ppmt = -np.ppmt(interest, np.arange(periods) + 1, periods, initial_principal)
    ipmt = -np.ipmt(interest, np.arange(periods) + 1, periods, initial_principal)
    return ppmt, ipmt

def print_mortgage(ppmt, ipmt):
    assert(len(ppmt) == len(ipmt))
    print(f"with a monthly payment of {(ppmt[0] + ipmt[0]) / 12}...")
    fmt = '{0:2d} {4:8.2f} {1:8.2f} {2:8.2f} {3:8.2f}'
    p = initial_principal
    per = np.arange(len(ppmt))
    for payment in per:
        index = payment - 1
        p = p - ppmt[index]
        print(fmt.format(payment, ppmt[index], ipmt[index], p, ppmt[index] + ipmt[index]))

### Gross Debt Service (GDS)

A GDS ratio is the percentage of your income needed to pay all of your monthly housing costs, including principal, interest, taxes, and heat (PITH). You’ll also need to include 50 per cent of your condo fees, if applicable.

### Total Debt Service Ratio

Includes other debt obligations (credit cards, lines of credit, car loans, etc.)

For rental property, you can include up to 50% of your rental income, and in this case, remove heating expenses from the calculation. But this would require at least a 20% down payment.

Canadian gov [reference](https://www.cmhc-schl.gc.ca/en/finance-and-investing/mortgage-loan-insurance/calculating-gds-tds)

Benchmark Interest Rate:
https://www.bankofcanada.ca/rates/daily-digest/

In [345]:
BENCHMARK_INTEREST_RATE=0.0519
def GDS(income, ppmt, ipmt, taxes, heat, condo_fees):
    return sum(p for p in ((ppmt + ipmt)[0], taxes, heat * 12, (condo_fees * 12)/2 )) / income

### Capital Tax

In [346]:
def capital_gains_tax(profit, income):
    federal, quebec = 0.15, 0.15
    if income > 43055:
        quebec = 0.20
    if income > 47630:
        federal = 0.205
    if income > 86105:
        quebec = 0.24
    if income > 95259:
        federal = 0.26
    if income > 104765:
        quebec = 0.2575
    if income > 147740:
        federal = 0.29
    if income > 210371:
        federal = 0.33
    
    # by law, only half of capital gains are taxed
    taxable_profit = profit / 2
    return (federal + quebec) * taxable_profit

### Comparative Yearly Return

Condominium appreciation 
[Plateau Mont Royale centris](https://www.centris.ca/en/tools/real-estate-statistics/montreal-island/le-plateau-mont-royal-montreal) -- 8% last year

[Montreal shupilov.com](https://news.shupilov.com/blog/average-real-estate-prices-and-appreciation-rates-in-montreal/) -- 3% per year

[fciq.ca](https://www.fciq.ca/pdf/mot_economiste/me_052014_an.pdf)

44% monthly cost salary as imposed by the CMHC insurance

In [347]:
def cumulative(np_array):
    return np.matmul(np_array, np.triu(np.ones(len(np_array) * len(np_array)).reshape((len(np_array), len(np_array)))))

def future_values(rate, nper, pmt, pv):
    return np.array([np.fv(rate, n, pmt, pv) for n in range(nper)])

def rental_income(start, incr, years):
    return np.array([12 * (starting_rent + i*constant_rent_increase) for i in range(nper)])

def net_operating_x(*np_arrays):
    return np.array([max(0, x) for x in sum(np_arrays)])

def realestate_value(appreciation, per, start):
    return future_values(appreciation, per, 0, -start)

def stocks(growth, capital, nper):
    average_pmt = np.average(capital[1:10])
    gains = future_values(growth, nper, -average_pmt, -capital[0])
    profit = gains - cumulative(capital)
    return gains - capital_gains_tax(profit, FUTURE_INCOME_AT_SELLTIME)

def realestate(price, appreciation, nper, pmt, ipmt, closing_cost, noi, nol):
    sell_price = realestate_value(appreciation, nper, price)
    gains = sell_price + cumulative(noi)
    losses = closing_cost * sell_price + cumulative(nol) + (sum(pmt) - cumulative(pmt))
    profit = gains - losses
    return gains - losses# - capital_gains_tax(profit, FUTURE_INCOME_AT_SELLTIME)

def calculate_rent_tax(yearly_rent, ipmt, *deductibles):
    # :ref https://www.revenuquebec.ca/en/citizens/your-situation/landlord-rental-property-owners/income-and-expenses/current-expenditures/
    # insurance, property tax, mortgage interest, maintenance, heating, electricity, water
    marginal_tax_rate = 0.44
    return (yearly_rent - ipmt - sum(deductibles)) * marginal_tax_rate

In [482]:
interest = 0.030
nper = 25
per = np.arange(nper) + 1
price = 375500
downpayment = 20000
condo_appreciation = 0.04
#starting_rent = 2220
#constant_rent_increase = 70
#rent = [12 * (starting_rent + i*constant_rent_increase) for i in range(nper)]
rent = np.array([2220, 2400] + [2400 + 40 * i for i in range(1, nper - 1)]) * 12

# To sell it
closing_cost = 0.04
FUTURE_INCOME_AT_SELLTIME = 100000

initial_capital = downpayment + INITIAL_EXPENSE
ppmt, ipmt = mortgage(price, downpayment, interest, 25)

income = rent
rental_tax = calculate_rent_tax(rent, ipmt, np.array([INSURANCE]*nper), np.array([PROPERTY_TAX]*nper), np.array([MAINTENANCE]*nper), np.array([110]*nper), np.array([CONDO_FEES] * nper))
losses = np.array([INITIAL_EXPENSE + ANNUAL_EXPENSE + downpayment] + [ANNUAL_EXPENSE] * (nper - 1)) + ppmt + ipmt + rental_tax
net = income - losses
noi = np.array([max(0, x) for x in net])
nol = -np.array([min(0, x) for x in net])

r = realestate(price, condo_appreciation, nper, ppmt, ipmt, closing_cost, noi, nol)
realestate_roi = r/cumulative(nol)
s = stocks(0.09, nol, nper)
stocks_roi = s/cumulative(nol)


(ANNUAL_EXPENSE + (ipmt + ppmt)[0]) / 12
GDS(78000, *mortgage(price, downpayment, BENCHMARK_INTEREST_RATE, 25), PROPERTY_TAX, 100, CONDO_FEES / 12)
rental_tax

array([ 3316.4274    ,  4399.74984619,  4747.85996577,  5100.07738893,
        5456.52533479,  5817.33071903,  6182.62426479,  6552.54061693,
        6927.21845963,  7306.80063761,  7691.43428093,  8081.27093355,
        8476.46668574,  8877.18231051,  9283.58340401,  9695.84053033,
       10114.12937043, 10538.63087573, 10969.53142619, 11407.02299317,
       11851.30330716, 12302.57603057, 12761.05093567, 13226.94408794,
       13700.47803477])

In [483]:
def display_comparison(s, r):
    print("year | stocks roi  | realestate roi | ratio | principal | interest | noi | rent after tax ")
    for i in range(len(s)):
        print(f"{i+1:4} | {s[i]:6.0f} {stocks_roi[i]:2.2f} | {r[i]:7.0f}  {realestate_roi[i]:5.2f} | {r[i]/s[i]:5.2f} | {ppmt[i]/12:9.0f} | {ipmt[i]/12:8.0f} | {net[i]:6.0f} | {(rent[i]-rental_tax[i]):4.0f}")

## Visualization

In [484]:
display_comparison(s, r)

year | stocks roi  | realestate roi | ratio | principal | interest | noi | rent after tax 
   1 |  37139 1.00 |  -33730  -0.91 | -0.91 |       839 |      918 | -37139 | 23324
   2 |  43943 1.05 |  -13601  -0.33 | -0.31 |       864 |      893 |  -4662 | 24400
   3 |  51221 1.11 |    7548   0.16 |  0.15 |       890 |      867 |  -4530 | 24532
   4 |  59020 1.16 |   29745   0.59 |  0.50 |       917 |      840 |  -4403 | 24660
   5 |  67391 1.22 |   53019   0.96 |  0.79 |       944 |      813 |  -4279 | 24783
   6 |  76389 1.29 |   77401   1.31 |  1.01 |       973 |      784 |  -4160 | 24903
   7 |  86076 1.36 |  102923   1.63 |  1.20 |      1002 |      755 |  -4045 | 25017
   8 |  96515 1.44 |  129618   1.93 |  1.34 |      1032 |      725 |  -3935 | 25127
   9 | 107779 1.52 |  157519   2.22 |  1.46 |      1063 |      694 |  -3830 | 25233
  10 | 119945 1.61 |  186662   2.50 |  1.56 |      1095 |      662 |  -3729 | 25333
  11 | 133099 1.70 |  217084   2.77 |  1.63 |      1128 |      629 | 